# Capítulo 03 — Self-Attention do Zero

Neste notebook vamos implementar, passo a passo, o mecanismo que tornou os Transformers e os LLMs modernos possíveis: **Self-Attention**.

Aqui vamos aprender:

- Como calcular pesos de atenção
- Como criar vetores de contexto
- Como surgem Query, Key e Value
- Como funciona máscara causal
- Como funciona Multi-Head Attention (conceitual)




In [ ]:
import torch
import torch.nn.functional as F

torch.manual_seed(42)


## 1 — Representando tokens como vetores

Vamos começar com embeddings simples.

Cada token da frase será representado como um vetor.


In [ ]:
tokens = ["Your", "journey", "starts", "with", "one", "step"]

embedding_dim = 3

embeddings = torch.rand(len(tokens), embedding_dim)

embeddings


## 2 - Atenção Intuitiva

Atenção pode ser entendida como uma média ponderada dos vetores de entrada.

Vamos calcular o contexto do segundo token ("journey").


In [ ]:
query = embeddings[1]

attention_scores = torch.matmul(embeddings, query)

attention_scores


Agora normalizamos os pesos usando softmax.


In [ ]:
attention_weights = F.softmax(attention_scores, dim=0)

attention_weights


Agora criamos o vetor de contexto.


In [ ]:
context_vector = torch.sum(attention_weights.unsqueeze(1) * embeddings, dim=0)

context_vector


## 3 - Self-Attention completo

Agora cada token calcula seu próprio contexto.

In [ ]:
def self_attention(inputs):

    scores = torch.matmul(inputs, inputs.T)
    weights = F.softmax(scores, dim=1)
    context = torch.matmul(weights, inputs)

    return context, weights

context_vectors, attention_matrix = self_attention(embeddings)

attention_matrix


## 4 - Query, Key e Value

Transformers não usam embeddings diretamente.

Eles criam três projeções treináveis:

- Query
- Key
- Value


In [ ]:
d_model = embeddings.shape[1]
d_k = 4

W_q = torch.rand(d_model, d_k)
W_k = torch.rand(d_model, d_k)
W_v = torch.rand(d_model, d_k)

Q = embeddings @ W_q
K = embeddings @ W_k
V = embeddings @ W_v

Q


## 5 - Self-Attention com QKV

In [ ]:
scores = Q @ K.T
scores = scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

weights = F.softmax(scores, dim=1)

context = weights @ V

context


## 6 - Máscara Causal

Modelos GPT não podem acessar tokens futuros.

In [ ]:
def causal_mask(size):
    return torch.triu(torch.ones(size, size), diagonal=1)

mask = causal_mask(len(tokens))
mask


In [ ]:
masked_scores = scores.masked_fill(mask == 1, float("-inf"))

masked_weights = F.softmax(masked_scores, dim=1)

masked_weights


7 - ## Dropout na Atenção

Dropout ajuda a evitar overfitting.

In [ ]:
dropout = torch.nn.Dropout(p=0.3)

dropped_weights = dropout(masked_weights)

dropped_weights


## 8 - Multi-Head Attention

Agora dividimos Q, K e V em múltiplas cabeças.

In [ ]:
def split_heads(x, num_heads):
    batch, dim = x.shape
    head_dim = dim // num_heads
    return x.view(batch, num_heads, head_dim)

num_heads = 2

Q_heads = split_heads(Q, num_heads)
K_heads = split_heads(K, num_heads)
V_heads = split_heads(V, num_heads)

Q_heads.shape


Agora aplicamos atenção em cada cabeça.


In [ ]:
heads_output = []

for i in range(num_heads):

    scores = Q_heads[:, i] @ K_heads[:, i].T
    weights = F.softmax(scores, dim=1)
    context = weights @ V_heads[:, i]

    heads_output.append(context)

multi_head_output = torch.cat(heads_output, dim=1)

multi_head_output


## 9 — Conectando com Transformers

### Onde isso entra no Transformer?

Self-Attention é o primeiro grande bloco da arquitetura GPT.

Ele permite que cada token compreenda o contexto global antes das camadas feed-forward.


## 10 - Conclusão

Você acabou de implementar o coração dos Transformers.

Você aprendeu:

✔ Weighted attention  
✔ Self-attention  
✔ QKV projections  
✔ Causal masking  
✔ Dropout  
✔ Multi-head attention  

No próximo capítulo vamos usar esse conhecimento para construir um GPT completo.


### 🎰 Bonus Stage

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(attention_matrix.detach(), cmap="viridis")
plt.colorbar()
plt.title("Attention Matrix")
plt.show()
